<a href="https://colab.research.google.com/github/SalmaSherif7070/Information-Retrieval-/blob/main/Project_DSAI_201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing

In [ ]:
!pip install nltk
# !pip install flair
!pip install python-terrier
!pip install flask pyngrok
!git clone https://github.com/terrierteam/terrier-prf/
!pip install transformers
!pip install --ignore-installed blinker
!pip install git+https://github.com/experimaestro/experimaestro-ir.git --quiet
!apt-get install maven   #used for Java projects to manage project dependencies and build processes
%cd /content/terrier-prf/
!mvn install
!pwd
%cd ..

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.9/337.9 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.2 MB/s eta 0:00:00
  Preparing metadata (setup

In [ ]:
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from gensim.scripts.glove2word2vec import glove2word2vec
# from flair.data import Sentence
# from flair.embeddings import WordEmbeddings
from termcolor import colored
import pandas as pd
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from scipy import spatial
import requests
from bs4 import BeautifulSoup
import re
import os
import tensorflow as tf
import tensorflow_hub as hub
import requests
from bs4 import BeautifulSoup
import torch
from xpmir.models import AutoModel
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity

elmo = hub.load("https://tfhub.dev/google/elmo/3")
nltk.download('punkt')
pd.set_option('display.max_colwidth', 150)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import pyterrier as pt
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

terrier-assemblies 5.9 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done
terrier-prf -SNAPSHOT jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8



# 1. Data collection

In [ ]:
url_list = ['https://www.theguardian.com/commentisfree/2023/oct/17/israel-palestine-hamas-gaza-west-leader-view',  # Topic 1
            'https://www.bbc.com/news/world-middle-east-20415675',
            'https://news.un.org/en/story/2024/04/1148876',
            'https://www.google.com/search?q=gaza+news+article&sca_esv=1dad59e041ee67d9&ei=UMcqZtT7Op-Nxc8P96i6sAE&oq=gaza+news+arti&gs_lp=Egxnd3Mtd2l6LXNlcnAiDmdhemEgbmV3cyBhcnRpKgIIADIFEAAYgAQyBhAAGBYYHjIGEAAYFhgeMgYQABgWGB4yCBAAGBYYChgeMgYQABgWGB4yCxAAGIAEGIYDGIoFMgsQABiABBiGAxiKBTILEAAYgAQYhgMYigUyCxAAGIAEGIYDGIoFSO0SUDBYpAtwAXgBkAEAmAGOAqABwQeqAQUwLjMuMrgBA8gBAPgBAZgCBqAC5wfCAgoQABiwAxjWBBhHwgINEAAYgAQYsAMYQxiKBcICDRAuGIAEGLADGEMYigXCAgoQABiABBhDGIoFmAMAiAYBkAYKkgcFMS4zLjKgB9gh&sclient=gws-wiz-serp#ip=1',
            'https://abcnews.go.com/International/live-updates/israel-gaza-hamas/?id=105171244',
            'https://www.aljazeera.com/news/liveblog/2024/4/25/israels-war-on-gaza-live-calls-grow-for-gaza-mass-graves-investigation',
            'https://www.aljazeera.com/program/al-jazeera-close-up/2024/4/25/how-gazas-children-prepare-for-israels-invasion-of-rafah-i-close-up',
            'https://www.understood.org/en/articles/personalized-learning-what-you-need-to-know', # Topic 2.1: Personalized learning
            'https://www.edweek.org/technology/what-is-personalized-learning/2019/11',
            'https://en.wikipedia.org/wiki/Personalized_learning',
            'https://elearningindustry.com/how-ai-is-personalizing-education-for-every-student',
            'https://www.valamis.com/hub/personalized-learning',
            'https://elearningindustry.com/from-one-size-fits-all-to-tailored-online-education-advantages-of-personalized-learning',
            'https://er.educause.edu/articles/2016/3/personalized-learning-what-it-really-is-and-why-it-really-matters',
            'https://elearningindustry.com/blended-learning-vs-traditional-learning-a-detailed-overview-of-the-two-approaches#:~:text=Traditional%20learning%2C%20often%20referred%20to,teachers%2C%20and%20interact%20with%20peers.',  # Topic 2.2: Traditional learning
            'https://www.igi-global.com/dictionary/designing-innovative-faculty-development-initiatives-through-the-lens-of-the-adult-learner/30346',
            'https://graphy.com/blog/traditional-method-of-teaching/',
            'https://www.celt.iastate.edu/instructional-strategies/teaching-format/traditional-face-to-face/',
            'https://potomac.edu/top-advantages-of-traditional-education/',
            'https://elearningindustry.com/traditional-learning-vs-online-learning',  # Online +trad
            'https://www.easy-lms.com/knowledge-center/e-learning/online-learning-vs-traditional-learning/item12530', # Online +trad
            "https://elearningindustry.com/what-is-online-learning-in-2023",    # Topic 2.2: Online learning
            'https://www.educations.com/study-guides/study-online',
            'https://elearningindustry.com/advantages-and-disadvantages-online-learning',
            'https://online-learning.com/',
            'https://www.udemy.com/',
            'https://www.coursera.org/'
            ]

data = []
for url in url_list:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    for script in soup(['script', 'style']):
        script.extract()
    text_content = soup.get_text()
    text_content = re.sub(r'\s+', ' ', text_content).strip()
    title = soup.title.string  # Extract the webpage title
    data.append({'url': url, 'title': title, 'text_content': text_content})

data


[{'url': 'https://www.theguardian.com/commentisfree/2023/oct/17/israel-palestine-hamas-gaza-west-leader-view',
  'title': 'Standing up for Palestine is also standing up to save the west from the worst of itself | Moustafa Bayoumi | The Guardian',
  'text_content': 'Standing up for Palestine is also standing up to save the west from the worst of itself | Moustafa Bayoumi | The Guardian Skip to main contentSkip to navigationClose dialogue1/1Next imagePrevious imageToggle captionSkip to navigationPrint subscriptions Sign inSearch jobsSearchUS editionUS editionUK editionAustralia editionEurope editionInternational editionThe Guardian - Back to homeThe GuardianNewsOpinionSportCultureLifestyleShowMoreShow MoreNewsView all NewsUS newsUS elections 2024Donald Trump trialsWorld newsEnvironmentUkraineSoccerBusinessTechScienceNewslettersWellnessOpinionView all OpinionThe Guardian viewColumnistsLettersOpinion videosCartoonsSportView all SportSoccerNFLTennisMLBMLSNBANHLF1GolfCultureView all CultureF

# 2. Preprocessing

In [ ]:
# preprocessing
nltk.download('stopwords')
stop_words = stopwords.words('english')
stemmer = PorterStemmer()

def preprocess(text):
  tokens = word_tokenize(text)
  tokens = [word.lower() for word in tokens if word.lower() not in stop_words]
  tokens = [stemmer.stem(word) for word in tokens]
  # print('Tokens are:',tokens,'\n')
  # return tokens
  return ' '.join(tokens)


df = pd.DataFrame(data)
df['docno'] = df.reset_index().index+1
df["docno"] = df["docno"].astype(str)
df['processed_text'] = df['text_content'].apply(preprocess)
df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,url,title,text_content,docno,processed_text
0,https://www.theguardian.com/commentisfree/2023/oct/17/israel-palestine-hamas-gaza-west-leader-view,Standing up for Palestine is also standing up to save the west from the worst of itself | Moustafa Bayoumi | The Guardian,Standing up for Palestine is also standing up to save the west from the worst of itself | Moustafa Bayoumi | The Guardian Skip to main contentSkip...,1,stand palestin also stand save west worst | moustafa bayoumi | guardian skip main contentskip navigationclos dialogue1/1next imageprevi imagetoggl...
1,https://www.bbc.com/news/world-middle-east-20415675,Gaza Strip in maps: How life has changed - BBC News,Gaza Strip in maps: How life has changed - BBC NewsBBC HomepageSkip to contentAccessibility HelpYour accountHomeNewsSportBusinessInnovationCulture...,2,gaza strip map : life chang - bbc newsbbc homepageskip contentaccess helpyour accounthomenewssportbusinessinnovationculturetravelmor menumor menus...
2,https://news.un.org/en/story/2024/04/1148876,"Mass graves in Gaza show victims’ hands were tied, says UN rights office | UN News","Mass graves in Gaza show victims’ hands were tied, says UN rights office | UN News Skip to main content / navigation Welcome to the United Nations...",3,"mass grave gaza show victim ’ hand tie , say un right offic | un news skip main content / navig welcom unit nation languag العربية 中文 english fran..."
3,https://www.google.com/search?q=gaza+news+article&sca_esv=1dad59e041ee67d9&ei=UMcqZtT7Op-Nxc8P96i6sAE&oq=gaza+news+arti&gs_lp=Egxnd3Mtd2l6LXNlcnAi...,gaza news article - Google Search,gaza news article - Google SearchGoogle×Please click here if you are not redirected within a few seconds. AllNewsImagesVideos Maps Shopping Books ...,4,gaza news articl - googl searchgoogle×pleas click redirect within second . allnewsimagesvideo map shop book search tool timeani timepast hourpast ...
4,https://abcnews.go.com/International/live-updates/israel-gaza-hamas/?id=105171244,Israel-Gaza updates: Israel says ground operation underway in southern Gaza - ABC News,Israel-Gaza updates: Israel says ground operation underway in southern Gaza - ABC News ABC NewsVideoLiveShowsElections538ShopInterest Successfully...,5,israel-gaza updat : israel say ground oper underway southern gaza - abc news abc newsvideoliveshowselections538shopinterest success addedw 'll not...
5,https://www.aljazeera.com/news/liveblog/2024/4/25/israels-war-on-gaza-live-calls-grow-for-gaza-mass-graves-investigation,"Israel’s war on Gaza updates: Evidence of torture, executions in mass grave | Israel War on Gaza News | Al Jazeera","Israel’s war on Gaza updates: Evidence of torture, executions in mass grave | Israel War on Gaza News | Al Jazeera Skip linksSkip to Contentplay L...",6,"israel ’ war gaza updat : evid tortur , execut mass grave | israel war gaza news | al jazeera skip linksskip contentplay live show navig menunavig..."
6,https://www.aljazeera.com/program/al-jazeera-close-up/2024/4/25/how-gazas-children-prepare-for-israels-invasion-of-rafah-i-close-up,How Gaza’s children are preparing for Israel’s invasion of Rafah I Close Up | Gaza | Al Jazeera,How Gaza’s children are preparing for Israel’s invasion of Rafah I Close Up | Gaza | Al Jazeera Skip linksSkip to Contentplay Live Show navigation...,7,gaza ’ children prepar israel ’ invas rafah close | gaza | al jazeera skip linksskip contentplay live show navig menunavig menunewsshow news secti...
7,https://www.understood.org/en/articles/personalized-learning-what-you-need-to-know,Personalized learning: What you need to know,Personalized learning: What you need to knowSkip to contentExplore resourcesPartner with usDonateExplore our resourcesBrowse by topicContent libra...,8,person learn : need knowskip contentexplor resourcespartn usdonateexplor resourcesbrows topiccont libraryhow spot sign kidspodcastsworkplac resour...
8,https://www.edweek.org/technology/what-is-personalized-learning/2019/11,What Is Personalized Learning?,What Is 

# 3. Indexing:

In [ ]:
indexer = pt.DFIndexer("./Path", overwrite = True)
indexref = indexer.index(df["processed_text"], df["docno"])
index = pt.IndexFactory.of(indexref)
print(index.getCollectionStatistics().toString())

Number of documents: 27
Number of terms: 4037
Number of postings: 10762
Number of fields: 0
Number of tokens: 27085
Field names: []
Positions:   false



In [ ]:
tf_idf = pt.BatchRetrieve(index, wmodel="TF_IDF")

# 5. Query expansion and Processing

In [ ]:
# rm3_expander = pt.rewrite.RM3(index,fb_terms=10, fb_docs=100)
# rm3_qe = tf_idf >> rm3_expander

In [ ]:
# new_retrieved_docs = {}
# for q in test_queries:
#   expanded_query = rm3_qe.search(q).iloc[0]["query"]
#   expanded_query_formatted = ' '.join(expanded_query.split()[1:])

#   results_wqe = tf_idf.search(expanded_query_formatted)
#   docnos = results_wqe['docno'].astype(int).tolist()
#   new_retrieved_docs[q] = docnos
#   # print(f"results of query {q}\n", results_wqe, '\n')
# # new_retrieved_docs

In [ ]:
def tf_idf_search(q, tf_idf, df):
    q = preprocess(q)
    results = tf_idf.search(q)
    result_merged = pd.DataFrame({'docno': results['docno'], 'score': results['score']})
    result_merged = result_merged.merge(df, on="docno")[["docno", "score", "processed_text"]]
    result_merged = result_merged.sort_values(by="score", ascending=False)
    return result_merged
tf_idf_search("learning", tf_idf, df)

,docno,score,processed_text
0,13,1.410554,advantag person learn cours join email list post articl advertis elearn industri dashboard portfolio bill profil sign login regist elearn industri...
1,12,1.409551,"person learn : plan , benefit , exampl [ guid ] skip content valami menu platformvalami platform learn platform help understand compani best learn..."
2,15,1.409364,blend learn tradit learn : comparison join email list post articl advertis elearn industri dashboard portfolio bill profil sign login regist elear...
3,8,1.407135,person learn : need knowskip contentexplor resourcespartn usdonateexplor resourcesbrows topiccont libraryhow spot sign kidspodcastsworkplac resour...
4,21,1.405670,onlin learn vs . tradit learn | easi lm skip content 're hire ! featur solut price client stori us demo helpen english ( unit state ) nederland fr...
5,24,1.398975,advantag disadvantag onlin learn - elearn industri join email list post articl advertis elearn industri dashboard portfolio bill profil sign login...
6,11,1.398345,ai person educ - elearn industri join email list post articl advertis elearn industri dashboard portfolio bill profil sign login regist elearn ind...
7,9,1.395460,person learn ? leadership back leadership budget & financ equiti & divers famili & commun profession develop recruit & retent school & district ma...
8,10,1.395010,person learn - wikipedia jump content main menu main menu move sidebar hide navig main pagecontentscurr eventsrandom articleabout wikipediacontact...
9,22,1.392533,onlin learn 2023 - elearn industri join email list post articl advertis elearn industri dashboard portfolio bill profil sign login regist elearn i...


In [ ]:
def elmo_search(q, tf_idf, df, elmo):
    result = tf_idf_search(q, tf_idf, df)
    result_data = [q] + [df['title'][int(docno)] for docno in result['docno'] if int(docno) < len(df)]
    embeddings = elmo.signatures["default"](tf.constant(result_data))["elmo"]
    sentence_embeddings = np.mean(embeddings.numpy(), axis=1)

    similarity_scores = []
    for i in range(1, len(result_data)):
        emb = sentence_embeddings[i]
        score = cosine_similarity(sentence_embeddings[0].reshape(1, -1), emb.reshape(1, -1))[0][0]
        similarity_scores.append((score, i))

    sorted_data = sorted(similarity_scores, key=lambda x: x[0], reverse=True)
    return pd.DataFrame(sorted_data, columns=['score', 'docno'])

elmo_search("learning", tf_idf, df, elmo)

,score,docno
0,0.655969,6
1,0.578265,8
2,0.432535,14
3,0.432332,4
4,0.415771,20
5,0.359719,3
6,0.350712,11
7,0.336160,17
8,0.328445,7
9,0.315734,2


In [ ]:
def elmo_berd_search(query, tf_idf, df, elmo):

        elmo_res = elmo_search(query, tf_idf, df, elmo)
        elmo_res['docno'] = elmo_res['docno'].astype(str)
        df['docno']=df['docno'].astype(str)

        elmo_res = elmo_res.merge(df, on="docno")[["docno", "score", "title"]]
        elmo_res = elmo_res.sort_values(by="score", ascending=False)
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        from transformers import AutoTokenizer, AutoModel
        bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
        bert_model = AutoModel.from_pretrained("bert-base-uncased")

        from xpmir.models import AutoModel
        model = AutoModel.load_from_hf_hub("xpmir/monot5", as_instance=True)

        output = model.rsv(query, elmo_res["title"].values)
        data = [(list(obj.document.items.values())[0].text, obj.score) for obj in output]
        result2 = pd.DataFrame(data, columns=['document', "score"]).sort_values(by="score", ascending=False)

        return result2['document'].tolist()


elmo_berd_search('learning', tf_idf, df, elmo)

['Personalized learning - Wikipedia',
 'Personalized Learning: What It Really Is and Why It Really Matters | EDUCAUSE Review',
 'Traditional Learning Vs. Online Learning - eLearning Industry',
 'What Is Personalized Learning?',
 'Personalized Learning: Plan, Benefits, Examples [Guide]',
 '\rWhat is Traditional Learning | IGI Global\r\n',
 'The Advantages Of Personalized Learning Courses',
 'Personalized learning: What you need to know',
 'Traditional Face-to-Face – Center for Excellence in Learning and Teaching',
 'Mass graves in Gaza show victims’ hands were tied, says UN rights office | UN News',
 'Top 6 Advantages of Traditional Education | University of the Potomac',
 'Blended Learning And Traditional Learning: A Comparison',
 'Israel’s war on Gaza updates: Evidence of torture, executions in mass grave | Israel War on Gaza News | Al Jazeera',
 'AI For Personalized Education - eLearning Industry',
 'gaza news article - Google Search',
 'What is the traditional method of teaching? | 

#7. Evaluation

In [ ]:
test_queries = ['Gaza', 'Personalized Learning', 'Learning', 'attack', 'Traditional learning', 'Online learning']
relevant_docs ={
    'Gaza': [1, 2, 3, 4, 5, 6, 7],
    'Personalized Learning': [8, 9, 10, 11, 12, 13, 14],
    'Learning': [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22],
    'attack': [1, 2, 3, 4, 5, 6, 7],
    'Traditional learning' :[15, 16, 17, 18, 19, 20, 21],
    'Online learning': [22, 23, 24, 25, 26]
}



In [ ]:
retrieved_docs = {}

for q in test_queries:
    docnos = elmo_berd_search(q, tf_idf, df, elmo)
    retrieved_docs[q] = docnos
    # print(f"results of query {q}\n", results, '\n')

print(retrieved_docs)

{'Gaza': ['5', '7', '4', '1', '3', '6', '2'], 'Personalized Learning': ['5', '3', '13', '2', '4', '7', '15', '6', '18', '8', '16', '14', '12', '11', '17', '9', '10', '1', '19', '21', '20', '22', '23'], 'Learning': ['6', '8', '4', '17', '11', '14', '3', '2', '7', '9', '20', '12', '13', '5', '18', '10', '16', '15', '1', '19'], 'attack': ['5', '1', '6', '2', '4', '3'], 'Traditional learning': ['13', '10', '4', '12', '6', '15', '20', '2', '16', '14', '1', '8', '17', '5', '18', '11', '3', '9', '7', '19'], 'Online learning': ['3', '14', '18', '5', '20', '9', '19', '15', '8', '12', '13', '16', '6', '1', '2', '10', '11', '4', '7', '17', '21']}


In [ ]:
precision_list = []
recall_list = []
f1_list = []
map_list = []
mrr_list = []
ndcg_list = []

for query in test_queries:
    relevant = relevant_docs.get(query, [])
    retrieved = retrieved_docs.get(query, [])[0]

    if len(retrieved) > 0:
        retrieved = [int(doc) for doc in retrieved]
        retrieved_set = set(retrieved)
    else:
        retrieved_set = set()

    # Precision
    relevant_set = set(relevant)
    precision = len(relevant_set & retrieved_set) / len(retrieved_set) if len(retrieved_set) > 0 else 0
    precision_list.append(precision)

    # Recall
    recall = len(relevant_set & retrieved_set) / len(relevant_set) if len(relevant_set) > 0 else 0
    recall_list.append(recall)

    # F1 Score
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    f1_list.append(f1)

    # MAP
    if len(relevant_set) > 0:
        avg_precision = np.mean([len(relevant_set & set(retrieved[:i+1])) / (i+1) for i in range(len(retrieved))])
    else:
        avg_precision = 0
    map_list.append(avg_precision)

    # nDCG
    dcg = 0
    for i in range(len(retrieved)):
        doc = retrieved[i]
        if doc in relevant_set:
            dcg += 1 / np.log2(i + 2)

    idcg = sum([1 / np.log2(i + 2) for i in range(len(relevant_set))])
    ndcg = dcg / idcg if idcg > 0 else 0
    ndcg_list.append(ndcg)


results = pd.DataFrame({
    'Query': test_queries,
    'Precision': precision_list,
    'Recall': recall_list,
    'F1 Score': f1_list,
    'MAP': map_list,
    'nDCG': ndcg_list
})

# print(results)


# Test:

In [ ]:
elmo_berd_search("Personalized Learning", tf_idf, df, elmo)

['Personalized Learning: What It Really Is and Why It Really Matters | EDUCAUSE Review',
 'Personalized Learning: Plan, Benefits, Examples [Guide]',
 'Personalized learning - Wikipedia',
 'Personalized learning: What you need to know',
 'What Is Personalized Learning?',
 'The Advantages Of Personalized Learning Courses',
 'AI For Personalized Education - eLearning Industry',
 'Online Learning in 2023 - What Is It And How Does It Work?',
 'Traditional Face-to-Face – Center for Excellence in Learning and Teaching',
 'Traditional Learning Vs. Online Learning - eLearning Industry',
 'What Is Online Learning In 2023 - eLearning Industry',
 'Israel’s war on Gaza updates: Evidence of torture, executions in mass grave | Israel War on Gaza News | Al Jazeera',
 'What is the traditional method of teaching? | Definition and Characteristics',
 'Mass graves in Gaza show victims’ hands were tied, says UN rights office | UN News',
 'Top 6 Advantages of Traditional Education | University of the Potomac